In [2]:
from models.FC_models import *

## helpers

In [3]:
import numpy as np

def imsave(img, name, path, saving_path=None):
    img = img / 2 + 0.5     # unnormalize
    #saving_path = os.path.join(args.output_path, name+'.png')
    print(f"saving to {saving_path }")
    torchvision.utils.save_image(img, saving_path)

def imshow(img, name, ax=None):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(10,10))
    if ax:
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))


In [5]:
PATCH_SIZE = 8

model_path = "saved_models/mikmik_fc_res-p8_b16-1_100.pkl"
model = Residual2CoreFC(coded_size=16,
                        patch_size=PATCH_SIZE,
                        num_passes=4)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

# training dataset

In [6]:
import torchvision
import torchvision.transforms as transforms

# Normalize function so given an image of range [0, 1] transforms it into a Tensor range [-1. 1]
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR LOADER
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)

dataiter = iter(train_loader)

Files already downloaded and verified


# produce reconstructed images

In [7]:
from evaluation import to_patches, reconstruct_patches

imgs, labels = next(dataiter)
print(labels[0])

#
def reconstruct(imgs, model = model):
    # break to patches
    patches = to_patches(imgs, PATCH_SIZE)
    r_patches = [model.sample(p) for p in patches] # pass all the patches
    reconstructed = reconstruct_patches(r_patches)

    return reconstructed

reconstructed = reconstruct(imgs,model)

tensor(99)


In [ ]:
import matplotlib.pyplot as plt

# show image alongside original
f, axarr = plt.subplots(1, 2)
imshow(reconstructed[0], "reconstructed", ax=axarr[0])
imshow(imgs[0], "original", ax=axarr[1])

# Classifier

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image

In [ ]:
classifier = torchvision.models.resnet18(num_classes=1000, weights=torchvision.models.ResNet18_Weights.DEFAULT)

In [ ]:
import torch.nn as nn

device = "cpu"
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        self.conv5 = conv_block(512, 1028, pool=True)
        self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))

        self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                        nn.Flatten(), # 1028
                                        nn.Linear(1028, num_classes)) # 1028 -> 100

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        out = self.classifier(out)
        return out

classifier = to_device(ResNet9(3, 100), device)
classifier

In [ ]:
# Initialize the model
#model = resnet100(pretrained=False)  # Set pretrained=False since we are loading custom weights

# Update this number of output features to match the CIFAR-100 classes if not using ResNet50 directly or the model has been modified
#model.fc = torch.nn.Linear(model.fc.in_features, 100)  # CIFAR-100 has 100 classes

# Load the weights
classifier.load_state_dict(torch.load('classifiers/group22_pretrained_model.h5',map_location=torch.device('cpu')))

# Set the model to evaluation mode
classifier.eval()

In [ ]:
imgs1, labels = next(dataiter)
print(labels[0])
print(f"predict from origin: {classifier(imgs1).argmax()}")

reconstructed = reconstruct(imgs1,model)
print(f"predicted from reconstruction: {classifier(reconstructed).argmax()}")

# show image alongside original
f, axarr = plt.subplots(1, 2)
imshow(reconstructed[0], "reconstructed", ax=axarr[0])
imshow(imgs1[0], "original", ax=axarr[1])


In [ ]:
def reconstruct_stages(imgs, model = model):
    # break to patches
    patches = to_patches(imgs, PATCH_SIZE)

    r_patches = [model.get_sampled_steps(p) for p in patches] # pass all the patches
    print(len(r_patches[0]))
    reconstructed = []
    for i in range(len(r_patches[0])):
        reconstructed.append( reconstruct_patches([r[i] for r in r_patches]))

    return reconstructed

reconstructed = reconstruct(imgs,model)

In [ ]:
stages = reconstruct_stages(imgs1)

f, axarr = plt.subplots(1, 5)
# imshow(reconstructed[0], "reconstructed", ax=axarr[0])
imshow(imgs1[0], "original", ax=axarr[0])

for i, stage in enumerate(stages):
    #reconstructed = reconstruct_patches(stage)
    imshow(stage[0], "original", ax=axarr[i+1])
